In [1]:
import pandas as pd
import numpy as np
import sys
import re
import os
import pandas as pd
sys.path.append("../automaterialsdata/")

from table_extractor import get_all_files_with_extension
from table_extractor import get_tables_from_xml_file
from table_extractor import gather_properties_from_full_table

In [2]:
Elsevier_xmls = get_all_files_with_extension('/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml', 'xml')
Elsevier_xmls = sorted(Elsevier_xmls, key=lambda x: int(x.split('/')[-1][:-4]))
Elsevier_xmls[:10]

['/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/68.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/114.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/127.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/135.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/141.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/146.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/151.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/155.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/174.xml',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/190.xml']

In [3]:
searching_patterns = [['VOC', 'VOC(V)'], 
                      ['JSC', 'Jsc(mAcm−2)','Avg.Jsc±std(mAcm−2)'], 
                      ['FF', 'FF(%)'],
                      ['PCE', 'PCE(%)', 'Avg.PCE±std(%)', 'Avg.PCE', 'Max.PCE']]
output_headers = ['V OC (V)', 'J SC (mA/cm-2)', 'FF (%)', 'PCE (%)'] 

all_properties = pd.DataFrame()
paper_count = -1
found_table_count = 0

for paper in Elsevier_xmls:
    
    paper_count += 1
    found_property_table = False
    table_count = -1
    paper_id = paper.split('/')[-1].split('.')[0]
    
    for table, table_header in zip(*get_tables_from_xml_file(paper)):
        table_count += 1
        
        properties_table = gather_properties_from_full_table(table, table_header, searching_patterns, output_headers)
        
        if properties_table is None: continue 
        found_property_table = True
        
        global_idx = []
        for row in range(len(properties_table.index)):
            to_append = str(paper_id) + '_' + str(table_count) + '_' + str(row) #Article Index + Table Index + Row Index 
            global_idx.append(to_append)
            
        properties_table.index = global_idx
        all_properties = pd.concat([all_properties, properties_table], sort=False)
        
    if not found_property_table:
        print(paper)
    else:
        found_table_count += 1
        
all_properties.to_csv('../data/Elsevier_XMLs_Table_Extractor_Reuslts.csv',encoding='utf-8')
print("found property tables in ", found_table_count, " papers (out of ", len(Elsevier_xmls), ")")
        

/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/Elsevier_xml/484.xml
found property tables in  46  papers (out of  47 )
